# Load and Preprocess

In [190]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [191]:
table_df = pd.read_csv("finaltable1.csv")

In [192]:
table_df.tail()

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average
375,375,497785,Newcastle Utd,Everton,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,14,3,5,6,14,21,-7,14,1.0,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,St James' Park,NaN,NaN,NaN,NaN,5.0,54.9756,-1.6217,2025-05-25T15:00:00,NaN,NaN,NaN,NaN,NaN,2025-05-25_Newcastle Utd_Everton,EVE,NEW,NaN,NaN,False,False,https://understat.com/match/26973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.0,38.0,2.0,4.0,Newcastle Utd,Everton,2025-05-25_Newcastle Utd_Everton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,376,497786,Nottingham Forest,Chelsea,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,9,4,2,35,18,17,31,2.6,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,The City Ground,NaN,NaN,NaN,NaN,6.0,52.9399,-1.1325,2025-05-25T15:00:00,NaN,NaN,NaN,NaN,NaN,2025-05-25_Nottingham Forest_Chelsea,CHE,NOT,NaN,NaN,False,False,https://understat.com/match/26972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,376.0,38.0,4.0,2.0,Nottingham Forest,Chelsea,2025-05-25_Nottingham Forest_Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,377,497787,Southampton,Arsenal,NaN,NaN,NaN,15,1,2,12,11,31,-20,5,0.2,15,8,5,2,29,15,14,29,2.2,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,St Mary's Stadium,NaN,NaN,NaN,NaN,15.0,50.9058,-1.3913,2025-05-25T15:00:00,NaN,NaN,NaN,NaN,NaN,2025-05-25_Southampton_Arsenal,ARS,SOU,NaN,NaN,False,False,https://understat.com/match/26977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377.0,38.0,5.0,2.0,Southampton,Arsenal,2025-05-25_Southampton_Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,378,497788,Tottenham,Brighton,NaN,NaN,NaN,15,6,2,7,31,19,12,20,0.8,15,6,6,3,25,22,3,24,1.6,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,Tottenham Hotspur Stadium,NaN,NaN,NaN,NaN,16.0,51.6047,-0.0664,2025-05-25T15:00:00,NaN,NaN,NaN,NaN,NaN,2025-05-25_Tottenham_Brighton,BRI,TOT,NaN,NaN,False,False,https://understat.com/match/26976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378.0,38.0,2.0,4.0,Tottenham,Brighton,2025-05-25_Tottenham_Brighton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,379,497789,Wolves,Brentford,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,7,2,6,31,28,3,23,2.0,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,Molineux Stadium,NaN,NaN,NaN,NaN,19.0,52.5903,-2.1303,2025-05-25T15:00:00,NaN,NaN,NaN,NaN,NaN,2025-05-25_Wolves_Brentford,BRE,WOL,NaN,NaN,False,False,https://understat.com/match/26978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379.0,38.0,2.0,2.0,Wolves,Brentford,2025-05-25_Wolves_Brentford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
len(table_df)

380

# Data Preprocessing

## Handling Missing Values

In [196]:
def check_missing_values():
    # Check for missing values in the dataset
    missing_values = table_df.isnull().sum()
    
    # Filter for columns with missing values
    missing_values = missing_values[missing_values > 0]
    
    # Display missing values
    print("Columns with Missing Values:")
    print(missing_values)


In [197]:
check_missing_values()

Columns with Missing Values:
Home Score             231
Away Score             231
Winner                 231
Day                     10
xG                     231
                      ... 
AY                     231
HR                     231
AR                     231
Home_player_average    210
Away_player_average    210
Length: 63, dtype: int64


Store null fixtures for using to prediction later

In [199]:
# Create a new DataFrame with rows where 'Home Score' or 'Away Score' is null
null_scores_df = table_df[table_df['Home Score'].isnull() | table_df['Away Score'].isnull()]


In [200]:
null_scores_df.head()

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average
140,140,497553,Everton,Liverpool,NaN,NaN,NaN,14,3,5,6,14,21,-7,14,1.0,14,11,2,1,29,11,18,35,2.6,2024-12-07,00:00,15,Sat,NaN,NaN,NaN,NaN,Goodison Park,NaN,NaN,NaN,NaN,4.0,53.4389,-2.9664,2024-12-07T00:00:00,NaN,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,LIV,EVE,NaN,NaN,False,False,https://understat.com/match/26748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
150,150,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Emirates Stadium,NaN,13.0,1.20,5.5,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,NaN,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,16.0,2.0,5.0,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.997956,5.174126
151,151,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Anfield,NaN,9.0,1.29,5.5,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,NaN,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.0,16.0,2.0,5.0,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.174799,5.263219
152,152,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,St James' Park,NaN,NaN,NaN,NaN,5.0,54.9756,-1.6217,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,NaN,2024-12-14_Newcastle Utd_Leicester City,LEI,NEW,NaN,NaN,False,False,https://understat.com/match/26760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0,16.0,2.0,4.0,Newcastle Utd,Leicester City,2024-12-14_Newcastle Utd_Leicester City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.365882,6.955506
153,153,497569,Wolves,Ipswich Town,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,1,6,8,14,27,-13,9,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Molineux Stadium,NaN,NaN,NaN,NaN,19.0,52.5903,-2.1303,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,NaN,2024-12-14_Wolves_Ipswich Town,IPS,WOL,NaN,NaN,False,False,https://understat.com/match/26758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.0,16.0,2.0,2.0,Wolves,Ipswich Town,2024-12-14_Wolves_Ipswich Town,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.508869,10.624012


Remove matches that have not been played..

In [202]:
table_df = table_df.dropna(subset=['Home Score', 'Away Score'])
# Reset the index after filtering
table_df.reset_index(drop=True, inplace=True)

In [203]:
check_missing_values()

Columns with Missing Values:
Away Win Odds                 149
Home Win Odds                 149
Draw Odds                     149
Temperature                    19
Precipitation                  19
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


In [204]:
# Fill missing 'xG' values with corresponding 'home_xg' values for the same row
table_df['xG'] = table_df.apply(
    lambda row: row['home_xg'] if pd.isnull(row['xG']) else row['xG'],
    axis=1
)
table_df['xG.1'] = table_df.apply(
    lambda row: row['away_xg'] if pd.isnull(row['xG.1']) else row['xG.1'],
    axis=1
)

# Verify that there are no more missing values in 'xG'
missing_xg_after = table_df['xG'].isnull().sum()
print(f"Remaining missing values in 'xG': {missing_xg_after}")
missing_xg_after = table_df['xG.1'].isnull().sum()
print(f"Remaining missing values in 'xG.1': {missing_xg_after}")


Remaining missing values in 'xG': 0
Remaining missing values in 'xG.1': 0


In [205]:
check_missing_values()

Columns with Missing Values:
Away Win Odds                 149
Home Win Odds                 149
Draw Odds                     149
Temperature                    19
Precipitation                  19
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


Drop the score column because we already have home scores and away scores

In [207]:
# Drop the 'Score' column from the dataset
table_df.drop(columns=['Score','Precipitation'], inplace=True)

In [208]:
check_missing_values()

Columns with Missing Values:
Away Win Odds                 149
Home Win Odds                 149
Draw Odds                     149
Temperature                    19
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


In [209]:
# Fill missing 'Attendance' values with the median attendance
median_attendance = table_df['Attendance'].median()
table_df['Attendance'].fillna(median_attendance, inplace=True)

# Verify there are no missing values in 'Attendance'
print(f"Remaining missing values in 'Attendance': {table_df['Attendance'].isnull().sum()}")


Remaining missing values in 'Attendance': 0


In [210]:
# Filter rows where 'Referee' is null
rows_with_missing_referee = table_df[table_df['Referee'].isnull()]
rows_with_missing_referee.head(7)

,Unnamed: 0,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average


In [211]:
# Fill missing 'Referee' values with the mode (most common referee)
mode_referee = table_df['Referee'].mode()[0]
table_df['Referee'].fillna(mode_referee, inplace=True)

# Verify there are no missing values in 'Referee'
print(f"Remaining missing values in 'Referee': {table_df['Referee'].isnull().sum()}")


Remaining missing values in 'Referee': 0


In [212]:
check_missing_values()

Columns with Missing Values:
Away Win Odds                 149
Home Win Odds                 149
Draw Odds                     149
Temperature                    19
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


In [213]:
from sklearn.impute import KNNImputer
import pandas as pd

# Load the dataset (assuming it's in table_df)
# Replace 'Temperature' and any other relevant columns for KNN imputation
columns_to_impute = ['Temperature']  # Include columns correlated with Temperature

# Select only the relevant columns for imputation
impute_df = table_df[columns_to_impute]

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 neighbors by default

# Perform KNN imputation
imputed_data = knn_imputer.fit_transform(impute_df)

# Replace the original 'Temperature' column with the imputed values
table_df['Temperature'] = imputed_data[:, 0]

# Verify the imputation
print("Remaining missing values in 'Temperature':", table_df['Temperature'].isnull().sum())


Remaining missing values in 'Temperature': 0


In [214]:
check_missing_values()

Columns with Missing Values:
Away Win Odds                 149
Home Win Odds                 149
Draw Odds                     149
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


In [215]:
columns_to_remove = ['Away Win Odds', 'Home Win Odds', 'Draw Odds', 'Unnamed: 0']
table_df.drop(columns=columns_to_remove, inplace=True)


In [216]:
check_missing_values()

Columns with Missing Values:
H2H_Home_Total_Wins_Last_4    142
H2H_Away_Total_Wins_Last_4    142
H2H_Draws_Last_4              142
dtype: int64


## Check for outliers

In [218]:
# Function to identify outliers based on IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numerical columns
numerical_columns = table_df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(table_df, col) for col in numerical_columns}

# Display the columns with outliers and their counts
outlier_counts = {col: len(outliers[col]) for col in outliers if len(outliers[col]) > 0}
print("Columns with outliers and their counts:")
print(outlier_counts)


Columns with outliers and their counts:
{'Home Score': 13, 'Away Score': 7, 'Prev Home Lost': 2, 'Prev Home Goal Difference': 3, 'Prev Away Won': 2, 'Prev Away Lost': 3, 'Prev Away Goals Conceded': 2, 'Prev Away Goal Difference': 6, 'Prev Away Points': 1, 'xG': 4, 'xG.1': 9, 'Temperature': 9, 'H2H_Home_Total_Wins_Last_4': 1, 'home_xg': 2, 'away_xg': 4, 'away_np_xg': 3, 'away_np_xg_difference': 4, 'away_ppda': 7, 'away_deep_completions': 6, 'home_np_xg': 2, 'home_np_xg_difference': 4, 'home_ppda': 8, 'home_deep_completions': 4, 'B365H': 13, 'B365D': 11, 'B365A': 9, 'HST': 1, 'AST': 2, 'AF': 2, 'HC': 1, 'AC': 1, 'HY': 1, 'AY': 1, 'HR': 10, 'AR': 12}


Append the first 10 upcoming fixtures

In [220]:
# Append the first 10 rows of null_scores_df to table_df using concat
table_df = pd.concat([table_df, null_scores_df.head(10)], ignore_index=True)

In [221]:
table_df.tail(12)

,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0_x,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Home_player_average,Away_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30,15,Sun,2.8,2.8,61184.0,Tottenham Hotspur Stadium,Anthony Taylor,16.0,51.6047,-0.0664,2024-12-08T16:30:00,15.206185,NaN,NaN,NaN,2024-12-08_Tottenham_Chelsea,CHE,TOT,3.25488,2.57326,True,True,https://understat.com/match/26745,3.0,0.9833,1.05092,-2.20396,7.958333,16.0,0.0,1.8101,3.25488,2.20396,9.857143,8.0,148.0,15.0,4.0,4.0,Tottenham,Chelsea,2024-12-08_Tottenham_Chelsea,Tottenham,Chelsea,3.4,3.9,2.00,5.0,8.0,17.0,11.0,5.0,10.0,2.0,2.0,0.0,0.0,7.610948,7.590750,NaN,NaN,NaN,NaN,NaN,NaN
148,497559,West Ham,Wolves,2.0,1.0,HOME_TEAM,14,4,3,7,18,27,-9,15,0.8,14,2,3,9,22,36,-14,9,1.4,2024-12-09,20:00,15,Mon,1.0,1.4,62474.0,London Stadium,John Brooks,7.0,51.5386,-0.0164,2024-12-09T20:00:00,15.206185,NaN,NaN,NaN,2024-12-09_West Ham_Wolves,WOL,WHU,1.19808,1.47567,True,True,https://understat.com/match/26746,0.0,1.5825,1.47567,0.27759,8.037037,6.0,3.0,1.1481,1.19808,-0.27759,9.043478,7.0,149.0,15.0,2.0,2.0,West Ham,Wolves,2024-12-09_West Ham_Wolves,West Ham,Wolves,1.9,3.9,3.75,4.0,5.0,12.0,17.0,11.0,0.0,5.0,4.0,0.0,0.0,11.030379,11.481568,NaN,NaN,NaN,NaN,NaN,NaN
149,497553,Everton,Liverpool,NaN,NaN,NaN,14,3,5,6,14,21,-7,14,1.0,14,11,2,1,29,11,18,35,2.6,2024-12-07,00:00,15,Sat,NaN,NaN,NaN,Goodison Park,NaN,4.0,53.4389,-2.9664,2024-12-07T00:00:00,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,LIV,EVE,NaN,NaN,False,False,https://understat.com/match/26748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,140.0,NaN,NaN,NaN,NaN,NaN
150,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Emirates Stadium,NaN,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,16.0,2.0,5.0,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.997956,5.174126,150.0,NaN,13.0,1.20,5.50,NaN
151,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Anfield,NaN,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.0,16.0,2.0,5.0,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.174799,5.263219,151.0,NaN,9.0,1.29,5.50,NaN
152,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,St James' Park,NaN,5.0,54.9756,-1.6217,2024-12-14T15:00:00,NaN,Na

# Feature Engineering

### Column names changed for feature engineering

In [224]:
import pandas as pd

data = table_df
# Mapping the old column names to new names where 'Home' and 'Away' are replaced with 'Team 1' and 'Team 2'
column_mapping = {
    'Home Team': 'Team 1',
    'Away Team': 'Team 2',
    'Home Score': 'Team 1 Score',
    'Away Score': 'Team 2 Score',
    'Prev Home Played': 'Prev Team 1 Played',
    'Prev Away Played': 'Prev Team 2 Played',
    'Prev Home Won': 'Prev Team 1 Won',
    'Prev Home Drawn': 'Prev Team 1 Drawn',
    'Prev Home Lost': 'Prev Team 1 Lost',
    'Prev Home Goals Scored': 'Prev Team 1 Goals Scored',
    'Prev Home Goals Conceded': 'Prev Team 1 Goals Conceded',
    'Prev Home Goal Difference': 'Prev Team 1 Goal Difference',
    'Prev Home Points': 'Prev Team 1 Points',
    'Prev Home Form': 'Prev Team 1 Form',
    'Prev Away Won': 'Prev Team 2 Won',
    'Prev Away Drawn': 'Prev Team 2 Drawn',
    'Prev Away Lost': 'Prev Team 2 Lost',
    'Prev Away Goals Scored': 'Prev Team 2 Goals Scored',
    'Prev Away Goals Conceded': 'Prev Team 2 Goals Conceded',
    'Prev Away Goal Difference': 'Prev Team 2 Goal Difference',
    'Prev Away Points': 'Prev Team 2 Points',
    'Prev Away Form': 'Prev Team 2 Form',
    'H2H_Home_Wins_Last_4': 'H2H_Team_1_Wins_Last_4',
    'H2H_Away_Wins_Last_4': 'H2H_Team_2_Wins_Last_4',
    'H2H_Draws_Last_4': 'H2H_Draws_Last_4',
    'home_team_code': 'team_1_code',
    'away_team_code': 'team_2_code',
    'home_xg': 'team_1_xg',
    'away_xg': 'team_2_xg',
    'home_team': 'team_1',
    'away_team': 'team_2',
    'HomeTeam': 'Team1',
    'AwayTeam': 'Team2',
    'home_team_fdr': 'team_1_fdr',
    'away_team_fdr': 'team_2_fdr',
    'home_deep_completions': 'team1_deep_completions',
    'away_deep_completions': 'team2_deep_completions',
    'away_np_xg': 'team2_np_xg',
    'home_np_xg': 'team1_np_xg',
    'away_np_xg_difference':'team2_np_xg_difference',
    'home_np_xg_difference':'home_np_xg_difference',
    'away_np_xg': 'team2_np_xg',
    'home_ppda':'team1_ppda', 
    'away_ppda': 'team2_ppda',
    'home_points': 'team1_points',
    'away_points': 'team2_points',
    'away_expected_points': 'team2_expected_points',
    'home_expected_points':'team1_expected_points',
    'home_np_xg_difference': 'team1_np_xg_difference',
    'Away_player_average': 'team2_player_average',
    'Home_player_average': 'team1_player_average',
    
}

# Renaming the columns in the dataframe
data.rename(columns=column_mapping, inplace=True)
data.columns


Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Home_Total_Wins_Last_4',
       'H2H_Away_Total_Wins_Last_4', 'H2H_Draws_Last_4', 'Composite Key',
       'team_2_code', 'team_1_code', 'team_1_xg', 'team_2_xg', 'is_result',
       'has_data', 'url', 'team2_points', 'team2_expected_points',
       'team2_np_xg', 'team2_np_xg

In [225]:
data.tail(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30,15,Sun,2.8,2.8,61184.0,Tottenham Hotspur Stadium,Anthony Taylor,16.0,51.6047,-0.0664,2024-12-08T16:30:00,15.206185,NaN,NaN,NaN,2024-12-08_Tottenham_Chelsea,CHE,TOT,3.25488,2.57326,True,True,https://understat.com/match/26745,3.0,0.9833,1.05092,-2.20396,7.958333,16.0,0.0,1.8101,3.25488,2.20396,9.857143,8.0,148.0,15.0,4.0,4.0,Tottenham,Chelsea,2024-12-08_Tottenham_Chelsea,Tottenham,Chelsea,3.4,3.9,2.00,5.0,8.0,17.0,11.0,5.0,10.0,2.0,2.0,0.0,0.0,7.610948,7.590750,NaN,NaN,NaN,NaN,NaN,NaN
148,497559,West Ham,Wolves,2.0,1.0,HOME_TEAM,14,4,3,7,18,27,-9,15,0.8,14,2,3,9,22,36,-14,9,1.4,2024-12-09,20:00,15,Mon,1.0,1.4,62474.0,London Stadium,John Brooks,7.0,51.5386,-0.0164,2024-12-09T20:00:00,15.206185,NaN,NaN,NaN,2024-12-09_West Ham_Wolves,WOL,WHU,1.19808,1.47567,True,True,https://understat.com/match/26746,0.0,1.5825,1.47567,0.27759,8.037037,6.0,3.0,1.1481,1.19808,-0.27759,9.043478,7.0,149.0,15.0,2.0,2.0,West Ham,Wolves,2024-12-09_West Ham_Wolves,West Ham,Wolves,1.9,3.9,3.75,4.0,5.0,12.0,17.0,11.0,0.0,5.0,4.0,0.0,0.0,11.030379,11.481568,NaN,NaN,NaN,NaN,NaN,NaN
149,497553,Everton,Liverpool,NaN,NaN,NaN,14,3,5,6,14,21,-7,14,1.0,14,11,2,1,29,11,18,35,2.6,2024-12-07,00:00,15,Sat,NaN,NaN,NaN,Goodison Park,NaN,4.0,53.4389,-2.9664,2024-12-07T00:00:00,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,LIV,EVE,NaN,NaN,False,False,https://understat.com/match/26748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-07_Everton_Liverpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,140.0,NaN,NaN,NaN,NaN,NaN
150,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Emirates Stadium,NaN,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,16.0,2.0,5.0,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.997956,5.174126,150.0,NaN,13.0,1.20,5.50,NaN
151,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Anfield,NaN,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,NaN,NaN,NaN,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.0,16.0,2.0,5.0,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.174799,5.263219,151.0,NaN,9.0,1.29,5.50,NaN
152,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,St James' Park,NaN,5.0,54.9756,-1.621

### FE: Average fdr

In [227]:
def calculate_fdr_cumulative_features(dataframe):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_fdr = {}
    games_played = {}
    
    # Create new columns for average FDR and cumulative FDR
    avg_team_1_fdr = []
    avg_team_2_fdr = []
    cum_team_1_fdr = []
    cum_team_2_fdr = []

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team_1 = row["Team 1"]
        team_2 = row["Team 2"]
        
        # Initialize if not present in dictionary
        if team_1 not in cumulative_fdr:
            cumulative_fdr[team_1] = 0
            games_played[team_1] = 0
        if team_2 not in cumulative_fdr:
            cumulative_fdr[team_2] = 0
            games_played[team_2] = 0

        # Update cumulative FDR
        cumulative_fdr[team_1] += row['team_1_fdr']
        cumulative_fdr[team_2] += row['team_2_fdr']
        
        # Increment games played
        games_played[team_1] += 1
        games_played[team_2] += 1

         # Record cumulative FDR for the current row
        cum_team_1_fdr.append(cumulative_fdr[team_1])
        cum_team_2_fdr.append(cumulative_fdr[team_2])
        
        # Calculate averages for teams before updating current match FDR
       
        avg_team_1_fdr.append(cumulative_fdr[team_1] / (games_played[team_1])) 
        avg_team_2_fdr.append(cumulative_fdr[team_2] / (games_played[team_2]))

    # Add new columns to the DataFrame
    dataframe['Cum Team 1 FDR'] = cum_team_1_fdr
    dataframe['Cum Team 2 FDR'] = cum_team_2_fdr
    dataframe['Avg Team 1 FDR'] = avg_team_1_fdr
    dataframe['Avg Team 2 FDR'] = avg_team_2_fdr

    

    return dataframe

# Calculate cumulative features excluding current match
detailed_fixtures = calculate_fdr_cumulative_features(data)

# Display selected rows with cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_fdr', 'team_2_fdr',
    'Cum Team 1 FDR', 'Cum Team 2 FDR', 'Avg Team 1 FDR', 'Avg Team 2 FDR'
]



In [228]:
detailed_fixtures[['Team 1','Team 2','Prev Team 1 Played',
'Prev Team 2 Played','Avg Team 1 FDR','Avg Team 2 FDR','team_1_fdr', 'team_2_fdr','Cum Team 1 FDR','Cum Team 2 FDR']].to_csv("detailed_fixture.csv")

In [229]:
table_df.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1.0,0.0,0.0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,1.0,2.0,3.0,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,3.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,NaN,NaN,NaN,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2.0,1.0,5.0,2.0,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0,4.475801,11.873978,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,5.0,2.0
2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3.0,0.0,0.0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3.0,1.0,2.0,5.0,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,2.0,5.0
3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,NaN,NaN,NaN,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4.0,1.0,2.0,2.0,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63,1.0,5.0,8.0,8.0,1.0,5.0,1.0,1.0,1.0,0.0,8.953171,8.830482,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,NaN,NaN,NaN,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,

In [230]:
table_df = table_df.drop(columns=['team_1', 'team_2','Team1', 'Team2'])

In [231]:
table_df.to_csv("fact_table.csv")

In [232]:
df = table_df

In [233]:
df.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Home_Total_Wins_Last_4',
       'H2H_Away_Total_Wins_Last_4', 'H2H_Draws_Last_4', 'Composite Key',
       'team_2_code', 'team_1_code', 'team_1_xg', 'team_2_xg', 'is_result',
       'has_data', 'url', 'team2_points', 'team2_expected_points',
       'team2_np_xg', 'team2_np_xg

### FE: Average ppda/deep completion

In [235]:
def calculate_weighted_ppda_deep_completions(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_ppda = {}
    cumulative_deep = {}
    cumulative_fdr = {}
    games_played = {}

    # Create new columns for weighted average PPDA and deep completions
    cum_ppda_team1 = []
    cum_ppda_team2 = []
    cum_deep_team1 = []
    cum_deep_team2 = []
    avg_ppda_team1 = []
    avg_ppda_team2 = []
    avg_deep_team1 = []
    avg_deep_team2 = []
    weighted_avg_ppda_team1 = []
    weighted_avg_ppda_team2 = []
    weighted_avg_deep_team1 = []
    weighted_avg_deep_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_ppda:
                cumulative_ppda[team] = 0
                cumulative_deep[team] = 0
                cumulative_fdr[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_ppda_team1.append(cumulative_ppda[team1])
        cum_ppda_team2.append(cumulative_ppda[team2])
        cum_deep_team1.append(cumulative_deep[team1])
        cum_deep_team2.append(cumulative_deep[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_ppda_team1.append(cumulative_ppda[team1] / games_played[team1])
            avg_deep_team1.append(cumulative_deep[team1] / games_played[team1])
            weighted_avg_ppda_team1.append((cumulative_ppda[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
            weighted_avg_deep_team1.append((cumulative_deep[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
        else:
            avg_ppda_team1.append(0)
            avg_deep_team1.append(0)
            weighted_avg_ppda_team1.append(0)
            weighted_avg_deep_team1.append(0)

        if games_played[team2] > 0:
            avg_ppda_team2.append(cumulative_ppda[team2] / games_played[team2])
            avg_deep_team2.append(cumulative_deep[team2] / games_played[team2])
            weighted_avg_ppda_team2.append((cumulative_ppda[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
            weighted_avg_deep_team2.append((cumulative_deep[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
        else:
            avg_ppda_team2.append(0)
            avg_deep_team2.append(0)
            weighted_avg_ppda_team2.append(0)
            weighted_avg_deep_team2.append(0)

        # Update cumulative PPDA, deep completions, and FDR after recording the current match
        cumulative_ppda[team1] += row['team1_ppda']
        cumulative_ppda[team2] += row['team2_ppda']
        cumulative_deep[team1] += row['team1_deep_completions']
        cumulative_deep[team2] += row['team2_deep_completions']
        cumulative_fdr[team1] += row['team_1_fdr']
        cumulative_fdr[team2] += row['team_2_fdr']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum PPDA Team 1'] = cum_ppda_team1
    df['Cum PPDA Team 2'] = cum_ppda_team2
    df['Avg PPDA Team 1'] = avg_ppda_team1
    df['Avg PPDA Team 2'] = avg_ppda_team2
    df['Cum Deep Completions Team 1'] = cum_deep_team1
    df['Cum Deep Completions Team 2'] = cum_deep_team2
    df['Avg Deep Completions Team 1'] = avg_deep_team1
    df['Avg Deep Completions Team 2'] = avg_deep_team2
    df['Weighted Avg PPDA Team 1'] = weighted_avg_ppda_team1
    df['Weighted Avg PPDA Team 2'] = weighted_avg_ppda_team2
    df['Weighted Avg Deep Completions Team 1'] = weighted_avg_deep_team1
    df['Weighted Avg Deep Completions Team 2'] = weighted_avg_deep_team2

    return df

# Example usage with your dataset
updated_data = calculate_weighted_ppda_deep_completions(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team1_ppda', 'team2_ppda',
    'Weighted Avg PPDA Team 1', 'Weighted Avg PPDA Team 2',
    'Weighted Avg Deep Completions Team 1', 'Weighted Avg Deep Completions Team 2'
]
display_df = updated_data[columns_to_display]


In [236]:
display_df.head(12)

,Match ID,Team 1,Team 2,team1_ppda,team2_ppda,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,7.379310,10.833333,0.000000,0.000000,0.0,0.0
1,497411,Ipswich Town,Liverpool,18.777778,8.739130,0.000000,0.000000,0.0,0.0
2,497412,Arsenal,Wolves,7.769231,10.818182,0.000000,0.000000,0.0,0.0
3,497413,Everton,Brighton,18.333333,7.916667,0.000000,0.000000,0.0,0.0
4,497414,Newcastle Utd,Southampton,16.250000,3.789474,0.000000,0.000000,0.0,0.0
5,497415,Nottingham Forest,Bournemouth,8.653846,9.954545,0.000000,0.000000,0.0,0.0
6,497416,West Ham,Aston Villa,9.333333,11.304348,0.000000,0.000000,0.0,0.0
7,497417,Brentford,Crystal Palace,12.666667,7.724138,0.000000,0.000000,0.0,0.0
8,497418,Chelsea,Manchester City,10.064516,15.058824,0.000000,0.000000,0.0,0.0
9,497419,Leicester City,Tottenham,20.687500,5.482759,0.000000,0.000000,0.0,0.0


In [237]:
updated_data.head(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1.0,0.0,0.0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,1.0,2.0,3.0,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,NaN,NaN,NaN,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2.0,1.0,5.0,2.0,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0,4.475801,11.873978,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,5.0,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,3.0,0.0,0.0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,3.0,1.0,2.0,5.0,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0,10.830699,6.027272,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,NaN,NaN,NaN,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,4.0,1.0,2.0,2.0,2024-08-17_Everton_Brighton,2.63,3.30,2.63,1.0,5.0,8.0,8.0,1.0,5.0,1.0,1.0,1.0,0.0,8.953171,8.830482,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,0.000000,0.000000,0.000000,0.000000,0.

In [238]:
updated_data.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost',
       ...
       'Avg PPDA Team 1', 'Avg PPDA Team 2', 'Cum Deep Completions Team 1',
       'Cum Deep Completions Team 2', 'Avg Deep Completions Team 1',
       'Avg Deep Completions Team 2', 'Weighted Avg PPDA Team 1',
       'Weighted Avg PPDA Team 2', 'Weighted Avg Deep Completions Team 1',
       'Weighted Avg Deep Completions Team 2'],
      dtype='object', length=103)

### FE: Avg xg stats

In [240]:
df.tail()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
154,497567,Nottingham Forest,Aston Villa,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,7,4,4,23,23,0,25,1.4,2024-12-14,17:30,16,Sat,NaN,NaN,NaN,The City Ground,NaN,6.0,52.9399,-1.1325,2024-12-14T17:30:00,NaN,NaN,NaN,NaN,2024-12-14_Nottingham Forest_Aston Villa,AVL,NOT,NaN,NaN,False,False,https://understat.com/match/26761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.0,16.0,3.0,2.0,2024-12-14_Nottingham Forest_Aston Villa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.232868,8.928523,154.0,NaN,2.5,2.40,2.75,NaN,47.0,44.0,2.9375,2.7500,234.715410,203.509756,15.647694,13.567317,92.0,120.0,6.133333,8.000000,45.899902,37.988488,17.991111,22.400000
155,497562,Brighton,Crystal Palace,NaN,NaN,NaN,15,6,6,3,25,22,3,24,1.6,15,2,7,6,14,20,-6,13,1.2,2024-12-15,14:00,16,Sun,NaN,NaN,NaN,The American Express Stadium,NaN,11.0,50.8618,-0.0833,2024-12-15T14:00:00,11.8290,NaN,NaN,NaN,2024-12-15_Brighton_Crystal Palace,CRY,BRI,NaN,NaN,False,False,https://understat.com/match/26756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,16.0,2.0,3.0,2024-12-15_Brighton_Crystal Palace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.078640,8.936591,155.0,NaN,NaN,NaN,NaN,0.0,51.0,48.0,3.1875,3.0000,155.532181,210.166351,10.368812,14.011090,104.0,90.0,6.933333,6.000000,33.871453,42.033270,22.648889,18.000000
156,497565,Manchester City,Manchester Utd,NaN,NaN,NaN,15,8,3,4,27,21,6,27,0.8,15,5,4,6,19,18,1,19,1.4,2024-12-15,16:30,16,Sun,NaN,NaN,NaN,Etihad Stadium,NaN,14.0,53.4831,-2.2003,2024-12-15T16:30:00,11.2595,NaN,NaN,NaN,2024-12-15_Manchester City_Manchester Utd,MUN,MCI,NaN,NaN,False,False,https://understat.com/match/26753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,16.0,3.0,5.0,2024-12-15_Manchester City_Manchester Utd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.697759,7.035769,156.0,NaN,NaN,NaN,NaN,0.0,48.0,49.0,3.0000,3.0625,167.388097,172.377026,11.159206,11.491802,207.0,105.0,13.800000,7.000000,33.477619,33.709285,41.400000,20.533333
157,497563,Chelsea,Brentford,NaN,NaN,NaN,15,9,4,2,35,18,17,31,2.6,15,7,2,6,31,28,3,23,2.0,2024-12-15,19:00,16,Sun,NaN,NaN,NaN,Stamford Bridge,NaN,9.0,51.4817,-0.1911,2024-12-15T19:00:00,11.0715,NaN,NaN,NaN,2024-12-15_Chelsea_Brentford,BRE,CHE,NaN,NaN,False,False,https://understat.com/match/26757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.0,16.0,2.0,4.0,2024-12-15_Chelsea_Brentford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.617343,7.459762,157.0,NaN,9.2,1.36,6.40,0.0,48.0,47.0,3.0000,2.9375,172.707411,193.965937,11.513827,12.9310

In [241]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    xg_stats = [
        'xG', 'xG.1',
    ]
    cumulative = {stat: {} for stat in xg_stats}
    games_played = {}

    # Initialize columns for cumulative, average, and weighted average stats
    for stat in xg_stats:
        df[f'Cum {stat}'] = 0
        df[f'Avg {stat}'] = 0.0
        df[f'Weighted Avg {stat}'] = 0.0

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in xg_stats:
                    cumulative[stat][team] = 0

        # Compute averages for each stat based on previous matches' data
        for stat in xg_stats:
            team_key = team1 if '.1' not in stat and 'team2' not in stat else team2
            corrected_stat = stat.replace('.1', '') if '.1' in stat else stat  # Adjust the stat key for team2

            # Before updating the cumulative, record the current game's value
            value_to_add = row[stat] if pd.notna(row[stat]) else 0

            # Calculate averages using prior match data
            if games_played[team_key] >= 1:  # Now checks if there's at least one past game
                current_avg = cumulative[corrected_stat][team_key] / games_played[team_key]
                df.at[index, f'Avg {stat}'] = current_avg
                fdr = row['Avg Team 1 FDR'] if team_key == team1 else row['Avg Team 2 FDR']
                df.at[index, f'Weighted Avg {stat}'] = current_avg * fdr
            else:
                df.at[index, f'Avg {stat}'] = 0
                df.at[index, f'Weighted Avg {stat}'] = 0

            # Update cumulative values after computing averages
            cumulative[corrected_stat][team_key] += value_to_add

        # Increment games played after updating stats
        games_played[team1] += 1
        games_played[team2] += 1

    return df

In [242]:
# Example usage with your dataset
data_df = calculate_weighted_xg_stats(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
display_df = data_df[columns_to_display]


In [243]:
display_df.tail()

,Match ID,Team 1,Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
154,497567,Nottingham Forest,Aston Villa,NaN,NaN,1.193333,1.666667,3.505417,4.583333
155,497562,Brighton,Crystal Palace,NaN,NaN,1.460000,1.306667,4.653750,3.920000
156,497565,Manchester City,Manchester Utd,NaN,NaN,1.913333,1.406667,5.740000,4.307917
157,497563,Chelsea,Brentford,NaN,NaN,2.133333,1.593333,6.400000,4.680417
158,497568,Southampton,Tottenham,NaN,NaN,1.060000,1.880000,3.577500,5.405000


In [244]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_xg = {}
    games_played = {}

    # Create new columns for xG stats
    cum_xg_team1 = []
    cum_xg_team2 = []
    avg_xg_team1 = []
    avg_xg_team2 = []
    weighted_avg_xg_team1 = []
    weighted_avg_xg_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_xg:
                cumulative_xg[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_xg_team1.append(cumulative_xg[team1])
        cum_xg_team2.append(cumulative_xg[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_xg_team1.append(cumulative_xg[team1] / games_played[team1])
            # Multiply the average by 'Avg Team 1 FDR' for weighted average
            weighted_avg_xg_team1.append(avg_xg_team1[-1] * row['Avg Team 1 FDR'])
        else:
            avg_xg_team1.append(0)
            weighted_avg_xg_team1.append(0)

        if games_played[team2] > 0:
            avg_xg_team2.append(cumulative_xg[team2] / games_played[team2])
            # Multiply the average by 'Avg Team 2 FDR' for weighted average
            weighted_avg_xg_team2.append(avg_xg_team2[-1] * row['Avg Team 2 FDR'])
        else:
            avg_xg_team2.append(0)
            weighted_avg_xg_team2.append(0)

        # Update cumulative xG after recording the current match
        cumulative_xg[team1] += row['team_1_xg']
        cumulative_xg[team2] += row['team_2_xg']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum xG Team 1'] = cum_xg_team1
    df['Cum xG Team 2'] = cum_xg_team2
    df['Avg xG Team 1'] = avg_xg_team1
    df['Avg xG Team 2'] = avg_xg_team2
    df['Weighted Avg xG Team 1'] = weighted_avg_xg_team1
    df['Weighted Avg xG Team 2'] = weighted_avg_xg_team2

    return df
    
# Example usage with your dataset
updated_data = calculate_weighted_xg_stats(data_df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
displayxg_df = updated_data[columns_to_display]


In [245]:
displayxg_df.tail(10)

,Match ID,Team 1,Team 2,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
149,497553,Everton,Liverpool,NaN,NaN,1.220985,2.409830,NaN,NaN,NaN,NaN,1.064286,2.028571,NaN,NaN
150,497561,Arsenal,Everton,NaN,NaN,2.195366,NaN,6.860519,NaN,NaN,NaN,1.706667,0.993333,5.333333,NaN
151,497564,Liverpool,Fulham,NaN,NaN,NaN,1.651905,NaN,4.749228,NaN,NaN,1.893333,1.373333,NaN,3.948333
152,497566,Newcastle Utd,Leicester City,NaN,NaN,1.402115,1.293649,4.206345,3.719242,NaN,NaN,1.433333,1.080000,4.300000,3.105000
153,497569,Wolves,Ipswich Town,NaN,NaN,1.088662,1.184411,3.334027,3.257131,NaN,NaN,1.020000,0.940000,3.123750,2.585000
154,497567,Nottingham Forest,Aston Villa,NaN,NaN,1.422717,2.038115,4.179232,5.604816,NaN,NaN,1.193333,1.666667,3.505417,4.583333
155,497562,Brighton,Crystal Palace,NaN,NaN,1.643623,1.498310,5.239047,4.494930,NaN,NaN,1.460000,1.306667,4.653750,3.920000
156,497565,Manchester City,Manchester Utd,NaN,NaN,2.104472,1.586753,6.313417,4.859432,NaN,NaN,1.913333,1.406667,5.740000,4.307917
157,497563,Chelsea,Brentford,NaN,NaN,2.260974,1.725643,6.782921,5.069075,NaN,NaN,2.133333,1.593333,6.400000,4.680417
158,497568,Southampton,Tottenham,NaN,NaN,1.262781,2.089459,4.261885,6.007195,NaN,NaN,1.060000,1.880000,3.577500,5.405000


In [246]:
def calculate_weighted_stats_xp(df):
    stats = ['np_xg', 'expected_points']  # Unified stat names
    cumulative = {stat: {} for stat in stats}  # Unified cumulative stats
    games_played = {}

    # Initialize result columns
    for stat in stats:
        df[f'Cum {stat}'] = 0  # Unified cumulative for each team
        df[f'Avg team1_{stat}'] = 0.0
        df[f'Avg team2_{stat}'] = 0.0
        df[f'Weighted Avg team1_{stat}'] = 0.0
        df[f'Weighted Avg team2_{stat}'] = 0.0

    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize teams in cumulative and games_played if not already present
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in stats:
                    cumulative[stat][team] = 0.0

        for stat in stats:
            # Process for team1
            if games_played[team1] > 0:  # Exclude current match stats
                avg_team1 = cumulative[stat][team1] / games_played[team1]
                fdr_team1 = row['Avg Team 1 FDR']
                weighted_avg_team1 = avg_team1 * fdr_team1

                df.at[index, f'Avg team1_{stat}'] = avg_team1
                df.at[index, f'Weighted Avg team1_{stat}'] = weighted_avg_team1
            else:
                df.at[index, f'Avg team1_{stat}'] = 0
                df.at[index, f'Weighted Avg team1_{stat}'] = 0

            # Process for team2
            if games_played[team2] > 0:  # Exclude current match stats
                avg_team2 = cumulative[stat][team2] / games_played[team2]
                fdr_team2 = row['Avg Team 2 FDR']
                weighted_avg_team2 = avg_team2 * fdr_team2

                df.at[index, f'Avg team2_{stat}'] = avg_team2
                df.at[index, f'Weighted Avg team2_{stat}'] = weighted_avg_team2
            else:
                df.at[index, f'Avg team2_{stat}'] = 0
                df.at[index, f'Weighted Avg team2_{stat}'] = 0

            # Update cumulative stats AFTER calculating averages
            team1_value = row[f'team1_{stat}'] if pd.notna(row[f'team1_{stat}']) else 0
            team2_value = row[f'team2_{stat}'] if pd.notna(row[f'team2_{stat}']) else 0

            cumulative[stat][team1] += team1_value
            cumulative[stat][team2] += team2_value

            # Unified cumulative (including current stats)
            df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]

        # Increment games played for both teams AFTER calculation
        games_played[team1] += 1
        games_played[team2] += 1

    return df


In [247]:
# Call the function to calculate stats
processed_data = calculate_weighted_stats_xp(data_df)

# Define the columns to display for checking
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2',
    'team1_np_xg', 'team2_np_xg', 'Avg team1_np_xg', 'Avg team2_np_xg',
    'team1_expected_points', 'team2_expected_points',  'Avg team1_expected_points','Avg team2_expected_points', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]

# Display the filtered DataFrame
processed_data[columns_to_display].tail(10) # Adjust the number of rows to inspect more games


/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_35742/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.461391' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_35742/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.8510999999999997' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]


,Match ID,Team 1,Team 2,team1_np_xg,team2_np_xg,Avg team1_np_xg,Avg team2_np_xg,team1_expected_points,team2_expected_points,Avg team1_expected_points,Avg team2_expected_points,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
149,497553,Everton,Liverpool,NaN,NaN,1.220985,2.192353,NaN,NaN,1.097971,2.235021,NaN,NaN,1.220985,2.409830,NaN,NaN,NaN,NaN,1.064286,2.028571,NaN,NaN
150,497561,Arsenal,Everton,NaN,NaN,2.093877,1.139586,NaN,NaN,1.929520,1.024773,NaN,NaN,2.195366,NaN,6.860519,NaN,NaN,NaN,1.706667,0.993333,5.333333,NaN
151,497564,Liverpool,Fulham,NaN,NaN,2.046196,1.550416,NaN,NaN,2.086020,1.675547,NaN,NaN,NaN,1.651905,NaN,4.749228,NaN,NaN,1.893333,1.373333,NaN,3.948333
152,497566,Newcastle Utd,Leicester City,NaN,NaN,1.300626,1.192161,NaN,NaN,1.234353,0.841720,NaN,NaN,1.402115,1.293649,4.206345,3.719242,NaN,NaN,1.433333,1.080000,4.300000,3.105000
153,497569,Wolves,Ipswich Town,NaN,NaN,1.088662,1.184411,NaN,NaN,0.915980,0.724433,NaN,NaN,1.088662,1.184411,3.334027,3.257131,NaN,NaN,1.020000,0.940000,3.123750,2.585000
154,497567,Nottingham Forest,Aston Villa,NaN,NaN,1.321228,1.936625,NaN,NaN,1.409567,1.761193,NaN,NaN,1.422717,2.038115,4.179232,5.604816,NaN,NaN,1.193333,1.666667,3.505417,4.583333
155,497562,Brighton,Crystal Palace,NaN,NaN,1.643623,1.447565,NaN,NaN,1.500007,1.286087,NaN,NaN,1.643623,1.498310,5.239047,4.494930,NaN,NaN,1.460000,1.306667,4.653750,3.920000
156,497565,Manchester City,Manchester Utd,NaN,NaN,2.053728,1.536009,NaN,NaN,1.678967,1.482680,NaN,NaN,2.104472,1.586753,6.313417,4.859432,NaN,NaN,1.913333,1.406667,5.740000,4.307917
157,497563,Chelsea,Brentford,NaN,NaN,2.108740,1.624153,NaN,NaN,1.678700,1.279107,NaN,NaN,2.260974,1.725643,6.782921,5.069075,NaN,NaN,2.133333,1.593333,6.400000,4.680417
158,497568,Southampton,Tottenham,NaN,NaN,1.148884,2.089459,NaN,NaN,0.710580,1.567553,NaN,NaN,1.262781,2.089459,4.261885,6.007195,NaN,NaN,1.060000,1.880000,3.577500,5.405000


### Save the data

In [249]:
processed_data.to_csv("updated_data.csv")

### FE Average win/draw/points/goalscored ... stats

In [251]:
import pandas as pd

def calculate_cumulative_stats(dataframe):
    # List of stats to track
    stats_columns = [
        'Won', 'Drawn', 'Lost', 'Goals Scored', 'Goals Conceded', 
        'Goal Difference', 'Points'
    ]

    # Ensure the necessary columns exist
    required_columns = [f'Prev Team 1 {stat}' for stat in stats_columns] + [f'Prev Team 2 {stat}' for stat in stats_columns]
    missing_columns = [col for col in required_columns if col not in dataframe.columns]
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {missing_columns}")

    # Initialize dictionaries to track cumulative stats and games played
    cumulative_stats = {}
    games_played = {}

    # Initialize columns in the DataFrame to store cumulative and average stats for each team
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = 0
        dataframe[f'Cum Team 2 {stat}'] = 0
        dataframe[f'Avg Team 1 {stat}'] = 0.0
        dataframe[f'Avg Team 2 {stat}'] = 0.0

    # Lists to store the values temporarily
    temp_cum_team1 = {stat: [] for stat in stats_columns}
    temp_cum_team2 = {stat: [] for stat in stats_columns}
    temp_avg_team1 = {stat: [] for stat in stats_columns}
    temp_avg_team2 = {stat: [] for stat in stats_columns}

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']

        if team1 not in cumulative_stats:
            cumulative_stats[team1] = {stat: 0 for stat in stats_columns}
            games_played[team1] = 0
        if team2 not in cumulative_stats:
            cumulative_stats[team2] = {stat: 0 for stat in stats_columns}
            games_played[team2] = 0

        for stat in stats_columns:
            # Update cumulative stats
            cumulative_stats[team1][stat] += row[f'Prev Team 1 {stat}']
            cumulative_stats[team2][stat] += row[f'Prev Team 2 {stat}']

        # Calculate averages using the count of games played so far
        for stat in stats_columns:
            if games_played[team1] > 0:
                avg_team_1_stat = cumulative_stats[team1][stat] / games_played[team1]
                temp_avg_team1[stat].append(avg_team_1_stat)
                temp_cum_team1[stat].append(cumulative_stats[team1][stat])
            else:
                temp_avg_team1[stat].append(0)
                temp_cum_team1[stat].append(0)
            
            if games_played[team2] > 0:
                avg_team_2_stat = cumulative_stats[team2][stat] / games_played[team2]
                temp_avg_team2[stat].append(avg_team_2_stat)
                temp_cum_team2[stat].append(cumulative_stats[team2][stat])
            else:
                temp_avg_team2[stat].append(0)
                temp_cum_team2[stat].append(0)

        # Increment games played after calculation
        games_played[team1] += 1
        games_played[team2] += 1

    # Assign lists to DataFrame columns
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = temp_cum_team1[stat]
        dataframe[f'Cum Team 2 {stat}'] = temp_cum_team2[stat]
        dataframe[f'Avg Team 1 {stat}'] = temp_avg_team1[stat]
        dataframe[f'Avg Team 2 {stat}'] = temp_avg_team2[stat]

    return dataframe

# Example usage assuming 'data' is your DataFrame loaded from 'your_data.csv'
try:
    updated_data = calculate_cumulative_stats(processed_data)
except ValueError as e:
    print(e)


In [252]:
updated_data[['Match ID', 'Team 1', 'Team 2'] + [col for col in updated_data.columns if 'Avg' in col or 'Cum' in col]].head()

,Match ID,Team 1,Team 2,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,2.0,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.461391,0.0,0.0,0.0,0.0,2.8511,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497411,Ipswich Town,Liverpool,5.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.271661,0.0,0.0,0.0,0.0,2.9820,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
2,497412,Arsenal,Wolves,2.0,5.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.204135,0.0,0.0,0.0,0.0,2.7690,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
3,497413,Everton,Brighton,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.196155,0.0,0.0,0.0,0.0,2.8365,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
4,497414,Newcastle Utd,Southampton,2.0,4.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.388319,0.0,0.0,0.0,0.0,2.8360,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0


In [253]:
updated_data.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1.0,0.0,0.0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,1.0,2.0,3.0,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.461391,0.0,0.0,0.0,0.0,2.8511,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,NaN,NaN,NaN,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2.0,1.0,5.0,2.0,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0,4.475801,11.873978,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.271661,0.0,0.0,0.0,0.0,2.9820,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,

#### Categorize date/time

In [255]:
data = updated_data

In [256]:
data.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1.0,0.0,0.0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,1.0,2.0,3.0,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.461391,0.0,0.0,0.0,0.0,2.8511,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,NaN,NaN,NaN,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2.0,1.0,5.0,2.0,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0,4.475801,11.873978,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.271661,0.0,0.0,0.0,0.0,2.9820,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,

In [257]:
import pandas as pd
from datetime import datetime

# Parse date and time columns
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')  # Corrected format
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time

# Function to determine sunset time based on season
def get_sunset_time(date):
    month = date.month
    if month in [12, 1, 2]:  # Winter
        return datetime.strptime("16:00", "%H:%M").time()
    elif month in [3, 4, 5]:  # Spring
        return datetime.strptime("20:30", "%H:%M").time() if month == 5 else datetime.strptime("19:30", "%H:%M").time() if month == 4 else datetime.strptime("18:30", "%H:%M").time()
    elif month in [6, 7, 8]:  # Summer
        return datetime.strptime("21:30", "%H:%M").time()
    elif month in [9, 10, 11]:  # Autumn
        return datetime.strptime("18:30", "%H:%M").time() if month == 9 else datetime.strptime("17:00", "%H:%M").time() if month == 10 else datetime.strptime("16:30", "%H:%M").time()

# Calculate sunset times
sunset_times = data['Date'].apply(get_sunset_time)

# Categorize match as daytime or nighttime
def categorize_match(match_time, sunset_time):
    return 'Nighttime' if match_time > sunset_time else 'Daytime'

match_categories = data.apply(lambda row: categorize_match(row['Time'], sunset_times.loc[row.name]), axis=1)

# Assign computed series to DataFrame all at once
data = data.assign(Sunset_Time=sunset_times, Match_Time_Category=match_categories)

# Display the resulting DataFrame
print(data[['Date', 'Time', 'Sunset_Time', 'Match_Time_Category']])


          Date      Time Sunset_Time Match_Time_Category
0   2024-08-16  19:00:00    21:30:00             Daytime
1   2024-08-17  11:30:00    21:30:00             Daytime
2   2024-08-17  14:00:00    21:30:00             Daytime
3   2024-08-17  14:00:00    21:30:00             Daytime
4   2024-08-17  14:00:00    21:30:00             Daytime
..         ...       ...         ...                 ...
154 2024-12-14  17:30:00    16:00:00           Nighttime
155 2024-12-15  14:00:00    16:00:00             Daytime
156 2024-12-15  16:30:00    16:00:00           Nighttime
157 2024-12-15  19:00:00    16:00:00           Nighttime
158 2024-12-15  19:00:00    16:00:00           Nighttime

[159 rows x 4 columns]


In [258]:
data.head(30)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,1.0,0.0,0.0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,1.0,2.0,3.0,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0,9.143500,8.974790,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.00,0,0.00,0.000000,0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.461391,0.000000,0.000000,0.000000,0.000000,2.8511,0.00000,0.00000,0.000000,0.000000,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,21:30:00,Daytime
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30:00,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,NaN,NaN,NaN,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,2.0,1.0,5.0,2.0,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0,4.475801,11.873978,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,5.000000,2.0000

#### PPG

In [260]:
import numpy as np

# Calculate points per game for Team 1
data['ppg_team1'] = np.where(
    data['Prev Team 1 Played'] > 0,
    data['Prev Team 1 Points'] / data['Prev Team 1 Played'],
    0
)

# Calculate points per game for Team 2
data['ppg_team2'] = np.where(
    data['Prev Team 2 Played'] > 0,
    data['Prev Team 2 Points'] / data['Prev Team 2 Played'],
    0
)


In [261]:
data.tail(30)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category,ppg_team1,ppg_team2
129,497534,Liverpool,Manchester City,2.0,0.0,HOME_TEAM,12,10,1,1,24,8,16,31,2.6,12,7,2,3,22,17,5,23,1.2,2024-12-01,16:00:00,13,Sun,3.4,0.8,60248.0,Anfield,Chris Kavanagh,20.0,53.4308,-2.9608,2024-12-01T16:00:00,11.907499,NaN,NaN,NaN,2024-12-01_Liverpool_Manchester City,MCI,LIV,3.700540,0.745015,True,True,https://understat.com/match/26726,0.0,0.0636,0.745015,-2.194355,15.687500,4.0,3.0,2.9016,2.939370,2.194355,17.352941,8.0,130.0,13.0,5.0,5.0,2024-12-01_Liverpool_Manchester City,2.10,3.60,3.30,7.0,2.0,9.0,8.0,7.0,4.0,1.0,3.0,0.0,0.0,8.140629,5.655442,NaN,NaN,NaN,NaN,NaN,NaN,37.0,40.0,2.846154,3.076923,114.916651,135.606658,9.576388,11.300555,124.0,177.0,10.333333,14.750000,25.537034,32.959952,27.555556,43.020833,0,1.925000,5.478846,0,2.008333,6.179487,27.809420,26.035780,2.317452,2.169648,6.595824,6.675841,54.484905,2.127159,2.106217,6.054222,6.480669,50.1999,2.210233,1.725992,6.290664,5.310744,65,57,5.416667,4.750000,4,15,0.333333,1.250000,9,6,0.750000,0.500000,151,176,12.583333,14.666667,34,84,2.833333,7.000000,117,92,9.750000,7.666667,199,186,16.583333,15.500000,16:00:00,Daytime,2.583333,1.916667
130,497545,Ipswich Town,Crystal Palace,0.0,1.0,AWAY_TEAM,13,1,6,6,13,24,-11,9,1.0,13,1,6,6,11,18,-7,9,1.2,2024-12-03,19:30:00,14,Tue,0.5,1.6,29533.0,Portman Road Stadium,Craig Pawson,2.0,52.0567,1.1450,2024-12-03T19:30:00,15.206185,NaN,NaN,NaN,2024-12-03_Ipswich Town_Crystal Palace,CRY,IPS,0.471708,1.654220,True,True,https://understat.com/match/26737,3.0,2.3167,1.654220,1.182512,13.56521

#### CUm Stats

In [263]:
# Initialize new columns for cumulative averages
stats = ['HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
for stat in stats:
    data[f'Cumulative_Avg_{stat}'] = 0.0

# Initialize dictionaries to track total stats and number of games for each team
team_stats_totals = {stat: {} for stat in stats}
team_game_counts = {stat: {} for stat in stats}

# Iterate through the DataFrame row by row
for index, row in data.iterrows():
    home_team = row['Team 1']
    away_team = row['Team 2']
    
    # Calculate cumulative averages for each stat
    for stat in stats:
        home_stat = stat  # Home version of the stat
        away_stat = stat  # Away version of the stat

        # Initialize stats for home and away teams if not already in dictionaries
        if home_team not in team_stats_totals[home_stat]:
            team_stats_totals[home_stat][home_team] = 0
            team_game_counts[home_stat][home_team] = 0
        if away_team not in team_stats_totals[away_stat]:
            team_stats_totals[away_stat][away_team] = 0
            team_game_counts[away_stat][away_team] = 0

        # Assign cumulative average to the new columns
        if team_game_counts[home_stat][home_team] > 0:
            data.at[index, f'Cumulative_Avg_{home_stat}'] = (
                team_stats_totals[home_stat][home_team] / team_game_counts[home_stat][home_team]
            )
        if team_game_counts[away_stat][away_team] > 0:
            data.at[index, f'Cumulative_Avg_{away_stat}'] = (
                team_stats_totals[away_stat][away_team] / team_game_counts[away_stat][away_team]
            )

        # Update totals and game counts with the current game's stats
        team_stats_totals[home_stat][home_team] += row[home_stat]
        team_game_counts[home_stat][home_team] += 1
        team_stats_totals[away_stat][away_team] += row[away_stat]
        team_game_counts[away_stat][away_team] += 1

# Display the updated DataFrame with cumulative averages
data.tail(13)


,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Total_Wins_Last_4,H2H_Away_Total_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,Unnamed: 0_x,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,team1_player_average,team2_player_average,Unnamed: 0,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum np_xg,Avg team1_np_xg,Avg team2_np_xg,Weighted Avg team1_np_xg,Weighted Avg team2_np_xg,Cum expected_points,Avg team1_expected_points,Avg team2_expected_points,Weighted Avg team1_expected_points,Weighted Avg team2_expected_points,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points,Sunset_Time,Match_Time_Category,ppg_team1,ppg_team2,Cumulative_Avg_HST,Cumulative_Avg_AST,Cumulative_Avg_HF,Cumulative_Avg_AF,Cumulative_Avg_HC,Cumulative_Avg_AC,Cumulative_Avg_HY,Cumulative_Avg_AY,Cumulative_Avg_HR,Cumulative_Avg_AR
146,497556,Leicester City,Brighton,2.0,2.0,DRAW,14,3,4,7,19,28,-9,13,0.8,14,6,5,3,23,20,3,23,1.4,2024-12-08,14:00:00,15,Sun,1.3,1.5,31647.0,King Power Stadium,Stuart Attwell,10.0,52.6203,-1.1422,2024-12-08T14:00:00,15.206185,NaN,NaN,NaN,2024-12-08_Leicester City_Brighton,BRI,LEI,1.42438,1.12184,True,True,https://understat.com/match/26751,1.0,1.0744,1.12184,-0.30254,12.423077,5.0,1.0,1.6399,1.42438,0.30254,14.500000,3.0,147.0,15.0,2.0,2.0,2024-12-08_Leicester City_Brighton,4.5,3.8,1.73,3.0,7.0,9.0,10.0,5.0,5.0,1.0,2.0,0.0,0.0,8.944019,11.054044,NaN,NaN,NaN,NaN,NaN,NaN,42.0,49.0,2.800000,3.266667,181.132016,143.109104,12.938001,10.222079,71.0,99.0,5.071429,7.071429,36.965718,34.316979,14.489796,23.739796,0,1.064286,2.980000,0,1.457143,4.760000,17.980360,23.532500,1.284311,1.680893,3.596072,5.490917,42.536751,1.175574,1.680893,3.291606,5.490917,35.1259,0.784707,1.530407,2.197180,4.999330,16,49,1.142857,3.500000,40,41,2.857143,2.928571,49,15,3.500000,1.071429,137,183,9.785714,13.071429,196,131,14.000000,9.357143,-59,52,-4.214286,3.714286,88,188,6.285714,13.428571,16:00:00,Daytime,0.928571,1.642857,4.928571,4.071429,11.785714,11.000000,6.071429,4.357143,2.285714,2.357143,0.142857,0.142857
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30:00,15,S

In [264]:
data.to_csv("fact_table.csv")